In [1]:
import pandas as pd
import sqlite3
import openpyxl
import csv
import re

In [14]:
def excel_to_sqlite(excel_file, db_file):
    # Connect to SQLite database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    cursor.execute("DROP TABLE IF EXISTS emissions")

    # Load the Excel workbook
    wb = openpyxl.load_workbook(excel_file, read_only=True)
    sheet = wb.active

    # Get column names from the first row
    columns = next(sheet.iter_rows(values_only=True))

    # Create table with dynamic column names
    create_table_sql = f'''CREATE TABLE IF NOT EXISTS emissions (
        {', '.join([f'"{col}" TEXT' for col in columns])}
    )'''
    cursor.execute(create_table_sql)

    # Prepare INSERT statement
    insert_sql = f'''INSERT INTO emissions VALUES (
        {', '.join(['?' for _ in columns])}
    )'''

    # Insert data
    for row in sheet.iter_rows(min_row=2, values_only=True):
        cursor.execute(insert_sql, row)

    conn.commit()
    conn.close()


def execute_query(db_file, query, fetch=True):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute(query)
    if fetch:
        results = cursor.fetchall()
    else:
        results = None
        conn.commit()
    conn.close()
    return results

def get_column_names(db_file):
    query = "PRAGMA table_info(emissions)"
    columns = execute_query(db_file, query)
    return [col[1] for col in columns]

def get_unique_values(db_file, column):
    query = f'SELECT DISTINCT "{column}" FROM emissions'
    return execute_query(db_file, query)

def count_unique_values(db_file, column):
    query = f'SELECT COUNT(DISTINCT "{column}") FROM emissions'
    return execute_query(db_file, query)[0][0]


In [3]:
# excel_file = 'Emissions_Totals_E_All_Data.xlsx'
db_file = 'emissions.db'

# excel_to_sqlite(excel_file, db_file)

# # Get column names
# columns = get_column_names(db_file)
# print("Columns in the table:", columns)

# # Example usage for each column
# for column in columns:co
#     print(f"\nColumn: {column}")
#     unique_values = get_unique_values(db_file, column)
#     print(f"Sample unique values (up to 5): {unique_values[:5]}")
#     unique_count = count_unique_values(db_file, column)
#     print(f"Number of unique values: {unique_count}")

# # Example of getting unique combinations of two columns
# if len(columns) >= 2:
#     query = f'SELECT DISTINCT "{columns[0]}", "{columns[1]}" FROM emissions LIMIT 5'
#     unique_combinations = execute_query(db_file, query)
#     print(f"\nSample unique combinations of {columns[0]} and {columns[1]}:")
#     for combo in unique_combinations:
#         print(combo)

def filter_columns(db_file):
    # Get current column names
    columns = execute_query(db_file, "PRAGMA table_info(emissions)")
    columns = [col[1] for col in columns]

    # Filter columns
    keep_columns = [col for col in columns if not col.startswith('Y') or col == 'Y2021']

    # Create new table with filtered columns
    create_table_sql = f'''CREATE TABLE emissions_filtered (
        {', '.join([f'"{col}" TEXT' for col in keep_columns])}
    )'''
    execute_query(db_file, create_table_sql, fetch=False)

    # Copy data to new table
    copy_data_sql = f'''INSERT INTO emissions_filtered 
        SELECT {', '.join([f'"{col}"' for col in keep_columns])}
        FROM emissions'''
    execute_query(db_file, copy_data_sql, fetch=False)

    # Drop old table and rename new table
    execute_query(db_file, "DROP TABLE emissions", fetch=False)
    execute_query(db_file, "ALTER TABLE emissions_filtered RENAME TO emissions", fetch=False)

# Use the function to filter columns
filter_columns(db_file)

In [4]:
def inspect_table(db_file, table_name, limit=5):
    # Get column names
    columns = execute_query(db_file, f"PRAGMA table_info({table_name})")
    column_names = [col[1] for col in columns]
    print(f"Columns in {table_name}:")
    print(", ".join(column_names))
    print("\nSample data:")

    # Get sample data
    sample_data = execute_query(db_file, f"SELECT * FROM {table_name} LIMIT {limit}")
    
    # Print sample data
    for row in sample_data:
        print(row)

    # Get row count
    row_count = execute_query(db_file, f"SELECT COUNT(*) FROM {table_name}")[0][0]
    print(f"\nTotal number of rows: {row_count}")

# Use the function to inspect the table
inspect_table(db_file, 'emissions')

Columns in emissions:


Sample data:


OperationalError: no such table: emissions

In [ ]:
execute_query(db_file, f"SELECT * FROM countryCode LIMIT 10")


In [6]:
execute_query(db_file, f"SELECT DISTINCT(Unit) FROM emissions")

OperationalError: no such table: emissions

### Trade Emissions Data

In [7]:
import sqlite3
import pandas as pd
import csv

def csv_to_sqlite(csv_file, db_file, table_name='tradeEmissions'):
    # Read CSV file using pandas, with missing values handled
    df = pd.read_csv(csv_file, encoding='utf-8', na_values=[''], keep_default_na=True)
    
    # Replace NaN with None for SQLite compatibility
    df = df.where(pd.notnull(df), None)

    # Connect to SQLite database
    conn = sqlite3.connect(db_file)
    
    # Write the data to a sqlite table
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    
    conn.close()
    print(f"Data imported successfully into table '{table_name}'")

    # Print some information about the imported data
    print(f"Number of rows imported: {len(df)}")
    print(f"Columns: {', '.join(df.columns)}")

    # Print first few rows
    print("\nFirst few rows of data:")
    print(df.head().to_string())

# Usage
csv_file = 'TradeData_10_17_2024_13_19_5_cleaned_dropped_zero_cost_countries.csv'
csv_to_sqlite(csv_file, db_file)

Data imported successfully into table 'tradeEmissions'
Number of rows imported: 19562
Columns: refYear, countryCode, countryISO, partnerCountryCode, partnerCountryISO, HSCode, cmdCode, motCode, Units, netWeight, grossWeight, cifvalue, costPerKg

First few rows of data:
   refYear countryCode countryISO partnerCountryCode partnerCountryISO  HSCode                                                   cmdCode    motCode Units  netWeight   grossWeight      cifvalue  costPerKg
0     2023         ARG  Argentina                W00             World   20130  Meat; of bovine animals, boneless cuts, fresh or chilled  TOTAL MOT    kg   69700.15  7.642760e+05  7.087275e+05  10.168235
1     2023         ARG  Argentina                ARG         Argentina   20130  Meat; of bovine animals, boneless cuts, fresh or chilled  TOTAL MOT    kg   69700.15  7.642760e+05  7.087275e+05  10.168235
2     2023         AUS  Australia                W00             World   20130  Meat; of bovine animals, boneless cuts

In [9]:
execute_query(db_file, f"""SELECT "motCode" FROM tradeEmissions""")

[('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('Air',),
 ('Air',),
 ('Air',),
 ('Air',),
 ('Air',),
 ('Sea',),
 ('Sea',),
 ('Sea',),
 ('Sea',),
 ('Road',),
 ('Road',),
 ('Road',),
 ('Road',),
 ('Road',),
 ('Road',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('Air',),
 ('Air',),
 ('Air',),
 ('Air',),
 ('Air',),
 ('Air',),
 ('Sea',),
 ('Sea',),
 ('Sea',),
 ('Sea',),
 ('Sea',),
 ('Sea',),
 ('Sea',),
 ('Sea',),
 ('Sea',),
 ('Sea',),
 ('Sea',),
 ('Road',),
 ('Road',),
 ('Road',),
 ('Road',),
 ('Road',),
 ('Road',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('TOTAL MOT',),
 ('Air',),
 ('Air',),
 ('S

## Geo Location Table

In [17]:
def csv_to_sqlite(csv_file, db_file, table_name='countryDistance'):
    # Read CSV file using pandas, with missing values handled
    df = pd.read_csv(csv_file, encoding='utf-8', na_values=[''], keep_default_na=True)
    
    # Replace NaN with None for SQLite compatibility
    df = df.where(pd.notnull(df), None)

    # Connect to SQLite database
    conn = sqlite3.connect(db_file)
    
    # Write the data to a sqlite table
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    
    conn.close()
    print(f"Data imported successfully into table '{table_name}'")

    # Print some information about the imported data
    print(f"Number of rows imported: {len(df)}")
    print(f"Columns: {', '.join(df.columns)}")

    # Print first few rows
    print("\nFirst few rows of data:")
    print(df.head().to_string())

# Usage
csv_file = 'country-distances.csv'
csv_to_sqlite(csv_file, db_file)

Data imported successfully into table 'countryDistance'
Number of rows imported: 235
Columns:  , ABW, AFG, AGO, AIA, ALB, AND, ARE, ARG, ARM, ASM, ATA, ATF, ATG, AUS, AUT, AZE, BDI, BEL, BEN, BES, BFA, BGD, BGR, BHR, BHS, BIH, BLM, BLR, BLZ, BMU, BOL, BRA, BRB, BTN, BWA, CAF, CAN, CCK, CHE, CHL, CHN, CIV, CMR, COD, COG, COK, COL, COM, CPV, CRI, CUB, CUW, CXR, CYM, CYP, CZE, DEU, DJI, DMA, DNK, DOM, DZA, ECU, EGY, ERI, ESH, ESP, EST, ETH, FIN, FJI, FRA, FRO, FSM, GAB, GBR, GEO, GGY, GHA, GIB, GIN, GLP, GMB, GNB, GNQ, GRC, GRD, GRL, GTM, GUF, GUM, GUY, HKG, HND, HRV, HTI, HUN, IDN, IMN, IND, IOT, IRL, IRN, IRQ, ISL, ISR, ITA, JAM, JEY, JOR, JPN, KAZ, KEN, KGZ, KHM, KIR, KNA, KWT, LBN, LBR, LBY, LCA, LIE, LKA, LSO, LTU, LUX, LVA, MAC, MAR, MCO, MDA, MDG, MDV, MEX, MHL, MKD, MLI, MLT, MMR, MNE, MNG, MNP, MOZ, MRT, MSR, MTQ, MUS, MWI, MYS, MYT, NAM, NCL, NER, NFK, NGA, NIC, NIU, NLD, NOR, NPL, NRU, NZL, OMN, PAK, PAN, PCN, PER, PHL, PLW, PNG, POL, PRI, PRT, PRY, PSE, PYF, QAT, REU, ROU, RUS

In [12]:
execute_query(db_file, f"""
    select ABW from countryDistance
    Limit 10""")

[(0.0,),
 (13252.562556919602,),
 (9500.724683275826,),
 (973.5351203846636,),
 (9085.425408426994,),
 (7565.414509724316,),
 (12767.098434098489,),
 (5360.114080711346,),
 (11105.217199030116,),
 (11491.1503575087,)]

## attach country codes to distance

In [19]:
def transform_distance_matrix(db_file):
    # First drop the normalized table if it exists
    drop_table_query = """
    DROP TABLE IF EXISTS normalized_distances;
    """
    execute_query(db_file, drop_table_query, fetch=False)
    
    # Create the normalized table with the correct structure
    create_table_query = """
    CREATE TABLE normalized_distances (
        origin_country TEXT,
        destination_country TEXT,
        distance FLOAT
    );
    """
    execute_query(db_file, create_table_query, fetch=False)
    
    # Get column names using pragma
    columns_query = """
    SELECT name FROM pragma_table_info('countryDistance');
    """
    columns = [row[0] for row in execute_query(db_file, columns_query, fetch=True)]
    columns = columns[1:]  # Remove the first column name since it's the row identifier
    
    # For each country pair, insert into normalized_distances
    for origin_country in columns:
        insert_query = f"""
        INSERT INTO normalized_distances (origin_country, destination_country, distance)
        SELECT 
            '{origin_country}' as origin_country,
            [0] as destination_country,
            [{origin_country}] as distance
        FROM countryDistance;
        """
        try:
            execute_query(db_file, insert_query, fetch=False)
        except Exception as e:
            print(f"Error inserting data for {origin_country}: {str(e)}")
            continue
    
    # Create index for better performance
    index_query = """
    CREATE INDEX idx_orig_dest ON normalized_distances(origin_country, destination_country);
    """
    execute_query(db_file, index_query, fetch=False)

def get_trade_emissions_with_distances(db_file):
    query = """
    SELECT 
        t.*,
        nd.distance
    FROM tradeEmissions t
    LEFT JOIN normalized_distances nd
        ON t.countryCode = nd.origin_country 
        AND t.partnerCountryCode = nd.destination_country;
    """
    return execute_query(db_file, query)

# Usage:
db_file = "your_database.db"  # Replace with your database file path

# First transform the matrix
transform_distance_matrix(db_file)

# Then get the results
results = get_trade_emissions_with_distances(db_file)

Error inserting data for ABW: no such column: 0
Error inserting data for AFG: no such column: 0
Error inserting data for AGO: no such column: 0
Error inserting data for AIA: no such column: 0
Error inserting data for ALB: no such column: 0
Error inserting data for AND: no such column: 0
Error inserting data for ARE: no such column: 0
Error inserting data for ARG: no such column: 0
Error inserting data for ARM: no such column: 0
Error inserting data for ASM: no such column: 0
Error inserting data for ATA: no such column: 0
Error inserting data for ATF: no such column: 0
Error inserting data for ATG: no such column: 0
Error inserting data for AUS: no such column: 0
Error inserting data for AUT: no such column: 0
Error inserting data for AZE: no such column: 0
Error inserting data for BDI: no such column: 0
Error inserting data for BEL: no such column: 0
Error inserting data for BEN: no such column: 0
Error inserting data for BES: no such column: 0
Error inserting data for BFA: no such co

OperationalError: no such table: tradeEmissions

In [22]:
countryDistance = pd.read_csv('country-distances.csv')
tradeEmission = pd.read_csv('TradeData_10_17_2024_13_19_5_cleaned_dropped_zero_cost_countries.csv')

In [24]:
def merge_trade_and_distances(trade_df, distance_df):
    # Convert distance matrix to long format
    long_distances = pd.melt(
        distance_df, 
        id_vars=distance_df.columns[0],  # First column contains country codes
        var_name='destination_country',
        value_name='distance'
    )
    
    # Rename the first column to 'origin_country'
    long_distances = long_distances.rename(columns={distance_df.columns[0]: 'origin_country'})
    
    # Merge trade data with distances
    result = trade_df.merge(
        long_distances,
        left_on=['countryCode', 'partnerCountryCode'],
        right_on=['origin_country', 'destination_country'],
        how='left'
    )
    
    return result

# Usage
result = merge_trade_and_distances(tradeEmission, countryDistance)

# Now result contains your trade emissions data with the distances added

In [81]:
country_result = result[~result['partnerCountryCode'].str.contains('\d', na = False)]
reduced_frame = result

,refYear,countryCode,countryISO,partnerCountryCode,partnerCountryISO,HSCode,cmdCode,motCode,Units,netWeight,grossWeight,cifvalue,costPerKg,distance
1,2023,ARG,Argentina,ARG,Argentina,20130,"Meat; of bovine animals, boneless cuts, fresh ...",TOTAL MOT,kg,69700.150,764276.010,708727.510,10.168235,0.0
3,2023,AUS,Australia,AUS,Australia,20130,"Meat; of bovine animals, boneless cuts, fresh ...",TOTAL MOT,kg,52778.780,832690.546,823195.459,15.597091,0.0
4,2023,AUS,Australia,JPN,Japan,20130,"Meat; of bovine animals, boneless cuts, fresh ...",TOTAL MOT,kg,111349.300,7954896.306,7044398.315,63.263966,7792.827411
5,2023,AUS,Australia,NZL,New Zealand,20130,"Meat; of bovine animals, boneless cuts, fresh ...",TOTAL MOT,kg,307969.000,2974549.422,2828523.346,9.184442,2161.0772232784225
7,2023,BOL,Bolivia (Plurinational State of),USA,USA,20130,"Meat; of bovine animals, boneless cuts, fresh ...",TOTAL MOT,kg,395.851,3922.000,3626.000,9.160012,6572.396345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19557,2023,ZMB,Zambia,IND,India,210390,Sauces and preparations therefor; mixed condim...,Other,kg,470.000,536.326,134.128,0.285379,6197.583155
19558,2023,ZMB,Zambia,ZAF,South Africa,210390,Sauces and preparations therefor; mixed condim...,Other,kg,3260.000,4031.164,1007.866,0.309161,2278.299176
19559,2023,ZMB,Zambia,ARE,United Arab Emirates,210390,Sauces and preparations therefor; mixed condim...,Other,kg,3610.000,4041.492,1010.389,0.279886,5369.146014
19560,2023,ZMB,Zambia,GBR,United Kingdom,210390,Sauces and preparations therefor; mixed condim...,Other,kg,10.000,173.815,43.489,4.348900,7908.901488


In [73]:
country_result.loc[:, 'distance'] = country_result['distance'].astype(float)

In [76]:
hold = country_result[country_result['motCode']=='TOTAL MOT']
hold[hold['distance']<2000]

,refYear,countryCode,countryISO,partnerCountryCode,partnerCountryISO,HSCode,cmdCode,motCode,Units,netWeight,grossWeight,cifvalue,costPerKg,origin_country,destination_country,distance
1,2023,ARG,Argentina,ARG,Argentina,20130,"Meat; of bovine animals, boneless cuts, fresh ...",TOTAL MOT,kg,69700.15,7.642760e+05,7.087275e+05,10.168235,ARG,ARG,0.000000
3,2023,AUS,Australia,AUS,Australia,20130,"Meat; of bovine animals, boneless cuts, fresh ...",TOTAL MOT,kg,52778.78,8.326905e+05,8.231955e+05,15.597091,AUS,AUS,0.000000
9,2023,BRA,Brazil,ARG,Argentina,20130,"Meat; of bovine animals, boneless cuts, fresh ...",TOTAL MOT,kg,1779901.00,1.917127e+07,1.892741e+07,10.633968,BRA,ARG,1673.973600
10,2023,BRA,Brazil,PRY,Paraguay,20130,"Meat; of bovine animals, boneless cuts, fresh ...",TOTAL MOT,kg,20859060.00,1.214491e+08,1.205179e+08,5.777723,BRA,PRY,1131.967396
11,2023,BRA,Brazil,URY,Uruguay,20130,"Meat; of bovine animals, boneless cuts, fresh ...",TOTAL MOT,kg,1165246.00,1.157388e+07,1.149069e+07,9.861174,BRA,URY,1562.322026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,2023,URY,Uruguay,ARG,Argentina,210390,Sauces and preparations therefor; mixed condim...,TOTAL MOT,kg,3806170.00,6.374444e+06,6.165827e+06,1.619956,URY,ARG,202.985729
19336,2023,URY,Uruguay,BRA,Brazil,210390,Sauces and preparations therefor; mixed condim...,TOTAL MOT,kg,939693.75,1.801113e+06,1.672798e+06,1.780152,URY,BRA,1562.322026
19337,2023,URY,Uruguay,CHL,Chile,210390,Sauces and preparations therefor; mixed condim...,TOTAL MOT,kg,53793.61,2.231207e+05,2.010499e+05,3.737431,URY,CHL,1341.490584
19357,2023,ZMB,Zambia,MWI,Malawi,210390,Sauces and preparations therefor; mixed condim...,TOTAL MOT,kg,930.00,1.831630e+02,4.581400e+01,0.049262,ZMB,MWI,613.406560


8258 for total MOT
9537 for NOT total mot

In [47]:
test=country_result[country_result['countryCode'] != country_result['origin_country']]

In [56]:
test['partnerCountryCode'].unique()

array(['KOR', 'LAO', '_X ', 'UMI', 'SXM', 'PRK', 'XX '], dtype=object)

In [57]:
test['countryCode'].unique()

array(['AUS', 'ARG', 'BOL', 'BRA', 'CHL', 'CZE', 'CAN', 'CPV', 'COL',
       'GUY', 'LUX', 'MUS', 'MDG', 'ISL', 'MEX', 'PHL', 'PRY', 'ROU',
       'MOZ', 'SVK', 'ZAF', 'SRB', 'USA', 'TUR', 'TZA', 'URY', 'ZMB'],
      dtype=object)

In [54]:
sum(country_result['motCode']=='AUS')

0

In [67]:
# Get list of unique values
print(country_result['partnerCountryCode'].unique())

# Get count of unique values
print(country_result['partnerCountryCode'].nunique())

# Get value counts (shows how many times each unique value appears)
print(country_result['partnerCountryCode'].value_counts())

# Get value counts as percentages
print(country_result['partnerCountryCode'].value_counts(normalize=True))

# Sort value counts in ascending order (default is descending)
print(country_result['partnerCountryCode'].value_counts().sort_index())

# If you want to see both counts and percentages:
counts = country_result['partnerCountryCode'].value_counts()
percentages = country_result['partnerCountryCode'].value_counts(normalize=True)
summary = pd.DataFrame({
    'count': counts,
    'percentage': percentages
})
print(summary)

['ARG' 'AUS' 'JPN' 'NZL' 'USA' 'PRY' 'URY' 'BRA' 'CAN' 'ITA' 'ESP' 'DNK'
 'DEU' 'ISL' 'IRL' 'NLD' 'SWE' 'GBR' 'CHL' 'MEX' 'BEL' 'FRA' 'JOR' 'MYS'
 'IND' 'VNM' 'GRC' 'AUT' 'BGR' 'CYP' 'HUN' 'IRN' 'LVA' 'NOR' 'POL' 'PRT'
 'ROU' 'CHE' 'MKD' 'EGY' 'CHN' 'PER' 'MMR' 'BOL' 'COL' 'HKG' 'KEN' 'NPL'
 'THA' 'PAK' 'KHM' 'LKA' 'BGD' 'IDN' 'IRQ' 'KOR' 'LAO' 'NGA' 'SAU' 'SGP'
 'ZAF' 'ARE' 'TUR' 'ISR' 'BIH' 'HRV' 'CZE' 'ECU' 'FJI' 'DJI' 'GEO' 'KWT'
 'LBN' 'LTU' 'LUX' 'MLT' 'MUS' 'MDA' 'OMN' 'NER' 'PHL' 'RUS' 'SRB' 'SYR'
 'UKR' 'WSM' 'GIB' 'EST' 'GTM' 'ARM' 'COK' 'CRI' 'JAM' 'MLI' 'TLS' 'HND'
 'BHR' 'BLZ' 'ETH' 'GHA' 'CIV' 'NIC' 'SLE' 'TUN' 'SLV' '_X ' 'FIN' 'MAR'
 'UMI' 'SVK' 'BWA' 'SVN' 'SEN' 'GIN' 'BTN' 'UGA' 'ALB' 'DOM' 'GAB' 'ASM'
 'KAZ' 'KGZ' 'UZB' 'DZA' 'AZE' 'BHS' 'CMR' 'COG' 'COD' 'BEN' 'GUY' 'LBR'
 'LBY' 'MDG' 'TGO' 'TZA' 'BFA' 'VEN' 'PYF' 'DMA' 'HTI' 'LCA' 'TTO' 'MOZ'
 'ZWE' 'SWZ' 'SUR' 'VCT' 'TKM' 'YEM' 'AFG' 'AND' 'AGO' 'ATG' 'BRB' 'BLR'
 'GMB' 'MAC' 'NAM' 'NCL' 'AIA' 'ZMB' 'FRO' 'TKL' 'P

## Tarrif Rates

In [ ]:
csv_file = 'tarrif_rates.csv'
csv_to_sqlite(csv_file, db_file, table_name="tarrif_rates")

In [ ]:
execute_query(db_file, f"""SELECT DISTINCT "Country Name" FROM tarrif_rates""")

In [ ]:
query_filter = f"""

SELECT 
    "Country Name", "Country Code", "Indicator Name", "Indicator Code", "2023"
FROM tarrif_rates
"""

execute_query(db_file, query_filter)

### FAO FOOD DATA

In [ ]:
execute_query(db_file, "DROP TABLE IF EXISTS FAOSTAT")

In [ ]:
csv_file = 'FAOSTAT_data.csv'
csv_to_sqlite(csv_file, db_file, table_name="FAOSTAT")

In [ ]:
execute_query(db_file, f"""SELECT * FROM FAOSTAT LIMIT 10""")